In [27]:
import random,math,argparse
import numpy as np
from numpy.random.mtrand import sample
from matplotlib import pyplot as plt
import networkx as nx

set_dag_size = [20,30,40,50,60,70,80,90]             #random number of DAG  nodes
set_max_out = [1,2,3,4,5]                              #max out_degree of one node
set_alpha = [0.5,1.0,2.0]                            #DAG shape
set_beta = [0.0,0.5,1.0,2.0]                         #DAG regularity

def DAGs_generate(mode = 'default',n = 10,max_out = 2,alpha = 1,beta = 1.0):
    ##############################################initialize###########################################
    mode = mode
    if mode != 'default':
        n = random.sample(set_dag_size,1)[0]
        max_out = random.sample(set_max_out,1)[0]
        alpha = random.sample(set_alpha,1)[0]
        beta = random.sample(set_alpha,1)[0]
    else:
        n = n
        max_out = max_out
        alpha = alpha
        beta = beta

    length = math.floor(math.sqrt(n)/alpha)
    mean_value = n/length
    random_num = np.random.normal(loc = mean_value, scale = beta,  size = (length,1))
    ###############################################division############################################
    position = {'Start':(0,4),'Exit':(10,4)}
    generate_num = 0
    dag_num = 1
    dag_list = []
    for i in range(len(random_num)):
        dag_list.append([])
        for j in range(math.ceil(random_num[i])):
            dag_list[i].append(j)
        generate_num += math.ceil(random_num[i])

    if generate_num != n:
        if generate_num<n:
            for i in range(n-generate_num):
                index = random.randrange(0,length,1)
                dag_list[index].append(len(dag_list[index]))
        if generate_num>n:
            i = 0
            while i < generate_num-n:
                index = random.randrange(0,length,1)
                if len(dag_list[index])==1:
                    i = i-1 if i!=0 else 0
                else:
                    del dag_list[index][-1]
                i += 1

    dag_list_update = []
    pos = 1
    max_pos = 0
    for i in range(length):
        dag_list_update.append(list(range(dag_num,dag_num+len(dag_list[i]))))
        dag_num += len(dag_list_update[i])
        pos = 1
        for j in dag_list_update[i]:
            position[j] = (3*(i+1),pos)
            pos += 5
        max_pos = pos if pos > max_pos else max_pos
        position['Start']=(0,max_pos/2)
        position['Exit']=(3*(length+1),max_pos/2)

    ############################################link###################################################
    into_degree = [0]*n
    out_degree = [0]*n
    edges = []
    pred = 0

    for i in range(length-1):
        sample_list = list(range(len(dag_list_update[i+1])))
        for j in range(len(dag_list_update[i])):
            od = random.randrange(1,max_out+1,1)
            od = len(dag_list_update[i+1]) if len(dag_list_update[i+1])<od else od
            bridge = random.sample(sample_list,od)
            for k in bridge:
                edges.append((dag_list_update[i][j],dag_list_update[i+1][k]))
                into_degree[pred+len(dag_list_update[i])+k]+=1
                out_degree[pred+j]+=1
        pred += len(dag_list_update[i])


    ######################################create start node and exit node################################
    for node,id in enumerate(into_degree):#给所有没有入边的节点添加入口节点作父亲
        if id ==0:
            edges.append(('Start',node+1))
            into_degree[node]+=1

    for node,od in enumerate(out_degree):#给所有没有出边的节点添加出口节点作儿子
        if od ==0:
            edges.append((node+1,'Exit'))
            out_degree[node]+=1

    #############################################plot##################################################
    return edges,into_degree,out_degree,position

In [28]:
with open('DAG_dataset_100k.txt', 'w') as f:
  x = (DAGs_generate(mode = 'default',n = 100000,max_out = 2,alpha = 1,beta = 1.0))
  f.write(str(x[0]))

<ipython-input-27-d00d903d8e5d>:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  for j in range(math.ceil(random_num[i])):
<ipython-input-27-d00d903d8e5d>:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  generate_num += math.ceil(random_num[i])


In [35]:
from collections import defaultdict, deque
import time

def topological_sort_kahn(dag):
    # Создаем словарь для хранения степеней входа каждой вершины
    in_degree = defaultdict(int)

    # Создаем словарь для хранения списка соседей каждой вершины
    neighbors = defaultdict(list)

    # Заполняем словари степеней входа и соседей
    for edge in dag:
        u, v = edge
        in_degree[v] += 1
        neighbors[u].append(v)

    # Добавляем начальные узлы, у которых нет входящих ребер
    queue = deque([u for u in neighbors.keys() if u not in in_degree])

    # Список для результата топологической сортировки
    result = []

    while queue:
        current_node = queue.popleft()
        result.append(current_node)

        for neighbor in neighbors[current_node]:
            in_degree[neighbor] -= 1

            if in_degree[neighbor] == 0:
                queue.append(neighbor)

    return result


In [36]:
with open("/content/DAG_dataset_100k.txt", "r") as file:
    content = file.read().strip()
    dag = eval(content)

start_time = time.time()

sorted_vertices = topological_sort_kahn(dag)

end_time = time.time()

execution_time = end_time - start_time

print(f"Топологическая сортировка: {sorted_vertices}")
print(f"Время выполнения: {execution_time:.6f} секунд")


Топологическая сортировка: ['Start', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 2

In [33]:
import multiprocessing as mp
from collections import defaultdict, deque
import time

def topological_sort_kahn_parallel(dag, processes=mp.cpu_count()):
    # Создаем словарь для хранения степеней входа каждой вершины
    in_degree = defaultdict(int)

    # Создаем словарь для хранения списка соседей каждой вершины
    neighbors = defaultdict(list)

    # Заполняем словари степеней входа и соседей
    for edge in dag:
        u, v = edge
        in_degree[v] += 1
        neighbors[u].append(v)

    # Добавляем начальные узлы, у которых нет входящих ребер
    queue = deque([u for u in neighbors.keys() if u not in in_degree])

    # Список для результата топологической сортировки
    result = []

    def worker(q, res):
        while q:
            current_node = q.popleft()
            res.append(current_node)

            for neighbor in neighbors[current_node]:
                in_degree[neighbor] -= 1

                if in_degree[neighbor] == 0:
                    q.append(neighbor)

    # Распределяем работу между процессами
    pool = mp.Pool(processes=processes)
    manager = mp.Manager()
    shared_queue = manager.Queue()
    shared_result = manager.list()

    for node in queue:
        shared_queue.put(node)

    jobs = [pool.apply_async(worker, args=(shared_queue, shared_result)) for _ in range(processes)]

    for job in jobs:
        job.wait()

    pool.close()
    pool.join()

    return list(shared_result)


In [34]:
with open("/content/DAG_dataset_100k.txt", "r") as file:
    content = file.read().strip()
    dag = eval(content)

start_time = time.time()

sorted_vertices = topological_sort_kahn_parallel(dag)

end_time = time.time()

execution_time = end_time - start_time

print(f"Топологическая сортировка: {sorted_vertices}")
print(f"Время выполнения: {execution_time:.6f} секунд")

Топологическая сортировка: []
Время выполнения: 0.525991 секунд
